# Gesture Volume Control

In [1]:
import cv2
import numpy as np
import time
import HandTrackingModule as htm
import math

In [2]:
# Webcam Weight and Height
wCam, hCam = 648, 488



cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)

pTime = 0

detector = htm.handDetector(detectionCon=0.7)

# PyCaw : https://github.com/AndreMiras/pycaw
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate( IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
# volume.GetMute()
# volume.GetMasterVolumeLevel()
volRange = volume.GetVolumeRange()

minVol = volRange[0]
maxVol = volRange[1]
vol = 0
volBar = 400
volPer = 0

while True:
    success, img = cap.read()

    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw=False)
    if len(lmList) != 0:
        #print(lmList[4], lmList[8])

        # [4] = Ibu jari, [8] = Telunjuk
        x1, y1 = lmList[4][1], lmList[4][2]
        x2, y2 = lmList[8][1], lmList[8][2]
        cx, cy = (x1+x2) // 2, (y1+y2) // 2

        # Menggambar bulatan pada ibu jari dan telunjuk
        cv2.circle(img, (x1,y1), 15, (255,0,255), cv2.FILLED)
        cv2.circle(img, (x2,y2), 15, (255,0,255), cv2.FILLED)
        # menambahkan garis yang menghubungkan antara 2 jari
        cv2.line(img, (x1,y1), (x2,y2), (255,0,255), 3)
        # menambahkan titik tengah pada garis
        cv2.circle(img, (cx, cy), 15, (255,0,255), cv2.FILLED)

        length = math.hypot(x2-x1, y2-y1)
        #print(length)

        # Hand range 10 - 200 : Hand range bisa berbeda-beda,
        # pakai print(length) untuk mengetahui hand range
        # Volume Range -65 - 0

        vol = np.interp(length, [10, 200], [minVol, maxVol])
        volBar = np.interp(length, [10, 200], [400, 150])
        volPer = np.interp(length, [10, 200], [0, 100])
        print(int(length), vol)
        volume.SetMasterVolumeLevel(vol, None)

        if length<50:
            cv2.circle(img, (cx, cy), 15, (0, 255, 0), cv2.FILLED)

    cv2.rectangle(img, (50, 150), (85, 400), (255,0,0), 3)
    cv2.rectangle(img, (50, int(volBar)), (85, 400), (255,0,0), cv2.FILLED)
    cv2.putText(img, f'{int(volPer)} %', (40,450), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 3)



    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    cv2.putText(img, f'FPS: {int(fps)}', (48,50), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 3)
    

    cv2.imshow("Image", img)
    cv2.waitKey(1)

144 -19.226809440526168
140 -20.504794821624756
143 -19.340209773249974
141 -20.11471120986807
140 -20.482768825942756
143 -19.49821056942762
146 -18.40263018444606
77 -41.93922805102001
149 -17.47300338189899
148 -17.801076388554186
186 -4.549007226682292
130 -23.99986574783047
138 -21.191425591708345
144 -19.15532195283678
140 -20.506018783307844
137 -21.615476803683336
138 -21.16784026076104
148 -17.764004750003984
144 -19.18866993141812
142 -19.820571667115715
137 -21.555379444027892
134 -22.562111097214313
133 -22.94599250298066
132 -23.32012230285141
128 -24.72095016299818
131 -23.696052631578944
123 -26.421090626450848
124 -26.094461310957094
130 -24.038152853546713
126 -25.41179513424106
123 -26.43783733952408
118 -28.159071167963788
125 -25.755205285136135
128 -24.724974321455704
125 -25.74421756925588
123 -26.40853494513938
134 -22.654258121910665
126 -25.30291299598006
135 -22.30202200848492
144 -19.22084820370757
138 -21.212538188855056
147 -18.144111357821686
153 -16.12283

KeyboardInterrupt: 